## Aggregation: valores estatísticos
---

In [5]:
import pandas as pd
import numpy as np

*aggregation* são valores estatístico de um conjunto de dados, como média, mediana, soma de todos os valores, etc. Pandas trabalha com os mesmo aggragators do numpy:
np.funções()|operação
---|---|
std|desvio padrão|
var|variância|
median|mediana|
percentile|porcentagem|
any|or|
all|and|

para series, os aggragators retornam um valor único:

In [6]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [7]:
ser.mean()

0.5623850983416314

In [8]:
ser.sum()

2.811925491708157

para datasets, por padrão, os aggregator retornam um valor para cada coluna:

In [10]:
df = pd.DataFrame({'A': rng.rand(5),'B': rng.rand(5)})
df

,A,B
0,0.183405,0.611853
1,0.304242,0.139494
2,0.524756,0.292145
3,0.431945,0.366362
4,0.291229,0.456070


In [11]:
df.mean()

A    0.347115
B    0.373185
dtype: float64

In [12]:
df.prod()

A    0.003683
B    0.004166
dtype: float64

especificando o eixo com `axis=`, é possível calcular para cada linha:

In [13]:
df.mean(axis=1)

0    0.397629
1    0.221868
2    0.408451
3    0.399153
4    0.373650
dtype: float64

In [14]:
df.prod(axis=1)

0    0.112217
1    0.042440
2    0.153305
3    0.158248
4    0.132821
dtype: float64

a função `describe()` apresenta os diversos resultados para aggregators para um dataset:

In [16]:
df.describe()

,A,B
count,5.000000,5.000000
mean,0.347115,0.373185
std,0.132773,0.176818
min,0.183405,0.139494
25%,0.291229,0.292145
50%,0.304242,0.366362
75%,0.431945,0.456070
max,0.524756,0.611853


In [17]:
ser.describe()

count    5.000000
mean     0.562385
std      0.308748
min      0.156019
25%      0.374540
50%      0.598658
75%      0.731994
max      0.950714
dtype: float64

outros aggregators próprios do pandas são:
pd.função()|operação
---|---|
count|total de itens
first|primeiro item
last|último item
mean|média
median|mediana
min|mínimo
max|máximo
std|desvio padrão
var|variância
mad|desvio médio absoluto

#### .groupby()
---

é uma forma de calcular os aggregates de um conjunto de informações através de um rótulo ou índice seguindo a ideia *split-apply-combine*:
1. o *split* involve a separação e o agrupamento de um dataframe dependendo do valor de uma chave passada pelo usuário;
2. o *apply* é a computação dos valores estatísticos;
3. o *combine* mescla os resultados e as apresentadas como um array.

In [18]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [21]:
dfg = df.groupby('key')
dfg

observe que é necessário passar o nome da coluna a qual deseja-se os cálculos, e o retorno da função é um objeto, não o resultado final. Isto é interessante porque, apenas quando, de fato, for solicitado o aggregate, é que seu resultado é apresentado:

In [22]:
dfg.sum()

,data
key,
A,3
B,5
C,7


In [23]:
dfg.prod()

,data
key,
A,0
B,4
C,10


In [24]:
dfg.count()

,data
key,
A,2
B,2
C,2


qualquer método de aggregation de pandas ou numpy é aceito pelo objeto `groupby`, bem como, qualquer operação suportada pelo series e dataframe.

inclusive, o objeto `groupby` pode receber indexing e slicing normalmente:

In [26]:
dfg_chave = dfg['key']
dfg_chave

e, da mesma forma, pode calcular os aggregators:

In [27]:
dfg_chave.sum()

key
A    AA
B    BB
C    CC
Name: key, dtype: object

objetos `groupby` também suportam iteração:

In [35]:
for i in dfg:
    print(i)

('A',   key  data
0   A     0
3   A     3)
('B',   key  data
1   B     1
4   B     4)
('C',   key  data
2   C     2
5   C     5)


é, inclusive, possível usar o describe junto do objeto `groupby`:

In [36]:
dfg.describe()

data                                         
    count mean      std  min   25%  50%   75%  max
key                                               
A     2.0  1.5  2.12132  0.0  0.75  1.5  2.25  3.0
B     2.0  2.5  2.12132  1.0  1.75  2.5  3.25  4.0
C     2.0  3.5  2.12132  2.0  2.75  3.5  4.25  5.0

sendo, este quadro, referente somente à coluna passada como parâmetro ao ser criado o objeto `groupby`.

o objeto `groupby` tem ainda uma variedade de métodos disponíveis que confere uma maior flexibilidade no cálculo das estatísticos do conjunto de dados.

o método `.aggregate()` pode recebe uma string, uma função, ou uma lista, por exemplo, e calcula todos os aggregators de uma só vez:

In [38]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],'data1': range(6),'data2': rng.randint(0, 10, 6)},columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [90]:
dfg = df.groupby('key')
dfg.aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

observe que as strings passadas em `.aggregate()` foram usadas para calculá-las em cada coluna.

se desejar especificar uma operação diferente para cada coluna, use dicionários:

In [91]:
dfg.aggregate({'data1': 'min', 'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


é possível filtrar os resultados usando o método `.filter()`, que recebe uma função que apresente algum critério de filtragem:

In [100]:
filter_func = lambda x: x['data2'].std() > 4
dfg.filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


o método `.transform()` faz alguma tranformação que o usuário deseja no objeto `groupby` original. também é necessário criar uma função separada para isso, que o método `.transform()` recebe como parâmetro:

In [101]:
double = lambda x: x + x
dfg.transform(double)

,data1,data2
0,0,10
1,2,0
2,4,6
3,6,6
4,8,14
5,10,18


o método `.apply()` também faz transformação, só que nos níveis mais do `groupby` original, não necessitando transformar todo o objeto:

In [107]:
coloquially = lambda x: x['data1'] + x['data2']**2
dfg.apply(coloquially)

key   
A    0    25
     3    12
B    1     1
     4    53
C    2    11
     5    86
dtype: int64

o mpetodo `.apply()` sempre retorna ou um objeto panda ou um valor numérico.